Aşama 1: Veri Hazırlığı ve Ön İşleme

In [66]:
!pip install faker

In [67]:
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker()


LOG_LINE_COUNT = 20


HTTP_METHODS = ['GET', 'POST', 'PUT', 'DELETE']
HTTP_STATUS_CODES = [200, 201, 301, 400, 401, 403, 404, 500, 502, 503]
URLS = [
    '/home',
    '/about',
    '/contact',
    '/products',
    '/products/item1',
    '/products/item2',
    '/api/data',
    '/login',
    '/signup'
]
USER_AGENTS = [
    fake.firefox,
    fake.chrome,
    fake.safari,
    fake.internet_explorer,
    fake.opera
]

def generate_log_line():
    ip_address = fake.ipv4()
    identity = '-'
    userid = '-'
    time = (datetime.now() - timedelta(seconds=random.randint(0, 86400))).strftime('%d/%b/%Y:%H:%M:%S %z')
    method = random.choice(HTTP_METHODS)
    resource = random.choice(URLS)
    protocol = 'HTTP/1.1'
    status_code = random.choice(HTTP_STATUS_CODES)
    response_size = random.randint(200, 5000)
    referrer = fake.url()
    user_agent = random.choice(USER_AGENTS)()

    log_line = f'{ip_address} {identity} {userid} [{time}] "{method} {resource} {protocol}" {status_code} {response_size} "{referrer}" "{user_agent}"'
    return log_line


with open('web_traffic.log', 'w') as log_file:
    for _ in range(LOG_LINE_COUNT):
        log_line = generate_log_line()
        log_file.write(log_line + '\n')



In [68]:
'''import pandas as pd
import re


with open("web_traffic.log") as file:
    log_lines = file.readlines()


log_entries = []


for line in log_lines:
    match = re.match(r'(\S+) - - \[(.*?)\] "(.*?)" (\d{3}) (\d+) "(.*?)" "(.*?)"', line)
    if match:
        log_entries.append({
            "ip": match.group(1),
            "timestamp": match.group(2),
            "request": match.group(3),
            "status_code": match.group(4),
            "response_size": match.group(5),
            "referrer": match.group(6),
            "user_agent": match.group(7),
        })


df_logs = pd.DataFrame(log_entries)
df_logs.head()
print(df_logs)'''




'import pandas as pd\nimport re\n\n\nwith open("web_traffic.log") as file:\n    log_lines = file.readlines()\n\n\nlog_entries = []\n\n\nfor line in log_lines:\n    match = re.match(r\'(\\S+) - - \\[(.*?)\\] "(.*?)" (\\d{3}) (\\d+) "(.*?)" "(.*?)"\', line)\n    if match:\n        log_entries.append({\n            "ip": match.group(1),\n            "timestamp": match.group(2),\n            "request": match.group(3),\n            "status_code": match.group(4),\n            "response_size": match.group(5),\n            "referrer": match.group(6),\n            "user_agent": match.group(7),\n        })\n\n\ndf_logs = pd.DataFrame(log_entries)\ndf_logs.head()\nprint(df_logs)'

In [69]:
print(df_logs.isnull().sum())

df_logs = df_logs.dropna()


df_logs["status_code"] = df_logs["status_code"].astype(int)


df_logs.head()


ip               0
timestamp        0
request          0
status_code      0
response_size    0
referrer         0
user_agent       0
dtype: int64


,ip,timestamp,request,status_code,response_size,referrer,user_agent
0,39.250.59.115,01/Sep/2024:01:16:37,PUT /api/data HTTP/1.1,404,3481,http://grant.org/,Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_9_4...
1,130.196.188.157,01/Sep/2024:13:41:24,POST /products/item2 HTTP/1.1,403,822,https://www.harrington.org/,Opera/8.99.(X11; Linux i686; ts-ZA) Presto/2.9...
2,114.78.148.71,01/Sep/2024:08:36:33,DELETE /products/item1 HTTP/1.1,404,1369,https://www.washington.com/,Mozilla/5.0 (X11; Linux x86_64; rv:1.9.7.20) G...
3,7.191.136.189,01/Sep/2024:10:07:39,DELETE /contact HTTP/1.1,400,2570,http://johnson-khan.com/,Mozilla/5.0 (compatible; MSIE 6.0; Windows NT ...
4,99.220.147.177,01/Sep/2024:07:05:39,PUT /contact HTTP/1.1,502,2224,https://hayes-cain.info/,Opera/9.55.(Windows NT 6.1; ce-RU) Presto/2.9....


In [70]:
df_logs.to_json("cleaned_web_traffic.json", orient="records", lines=True)



In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer


df_logs["combined"] = (
    df_logs["ip"] + " " +
    df_logs["timestamp"] + " " +
    df_logs["request"] + " " +
    df_logs["status_code"].astype(str) + " " +
    df_logs["response_size"].astype(str) + " " +
    df_logs["referrer"] + " " +
    df_logs["user_agent"]
)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_logs["combined"])


print(X.shape)
print(X)


(20, 252)
  (0, 234)	0.11533204495639439
  (0, 243)	0.11533204495639439
  (0, 185)	0.09237394394711596
  (0, 208)	0.11533204495639439
  (0, 206)	0.11533204495639439
  (0, 47)	0.30405846719751217
  (0, 124)	0.36784305231331865
  (0, 163)	0.11533204495639439
  (0, 251)	0.18392152615665933
  (0, 242)	0.209235862885647
  (0, 233)	0.14064638168538204
  (0, 16)	0.209235862885647
  (0, 224)	0.12268557394309244
  (0, 211)	0.12268557394309244
  (0, 227)	0.16596071841436974
  (0, 212)	0.15202923359875609
  (0, 216)	0.07562546238083835
  (0, 223)	0.18392152615665933
  (0, 188)	0.209235862885647
  (0, 89)	0.209235862885647
  (0, 103)	0.15202923359875609
  (0, 192)	0.12486567264493031
  (0, 179)	0.15202923359875609
  (0, 162)	0.15202923359875609
  (0, 230)	0.15202923359875609
  :	:
  (19, 215)	0.13673095407322486
  (19, 199)	0.1613405698429927
  (19, 178)	0.1613405698429927
  (19, 200)	0.1613405698429927
  (19, 110)	0.17880137010746378
  (19, 164)	0.11927015380875378
  (19, 87)	0.10572650492373249


In [73]:
!pip install faiss-cpu


In [74]:
import faiss


index = faiss.IndexFlatL2(X.shape[1])


index.add(X.toarray())


faiss.write_index(index, "faiss_index.index")


In [75]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize_query(query, vectorizer):
    return vectorizer.transform([query]).toarray()


def retrieve_top_k(query_vector, index, k=1):
    D, I = index.search(query_vector, k)
    return I[0], D[0]


user_query = "Hangi sayfalar 500  status kodunu almıştır ?"


query_vector = vectorize_query(user_query, vectorizer)


indices, distances = retrieve_top_k(query_vector, index)


matching_logs = df_logs.iloc[indices]
matching_logs


,ip,timestamp,request,status_code,response_size,referrer,user_agent,combined
5,31.65.65.108,31/Aug/2024:17:46:03,POST /products HTTP/1.1,500,4229,https://taylor.com/,Opera/8.46.(X11; Linux x86_64; brx-IN) Presto/...,31.65.65.108 31/Aug/2024:17:46:03 POST /produ...


Aşama 2: RAG Modelinin Kurulumu

In [76]:
!pip install transformers
!pip install torch


In [77]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd


model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

df_logs = pd.DataFrame(log_entries)


def generate_response(logs, user_query):

    prompt = f"Kullanıcı sorusu: {user_query}\n"
    prompt += "İlgili log kayıtları:\n"
    for log in logs.to_dict(orient="records"):
        prompt += f"{log['timestamp']} - {log['request']} - {log['status_code']}\n"
    prompt += "\nBu bilgilere dayanarak, kullanıcı sorusuna yanıt verin."


    inputs = tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=512)


    outputs = model.generate(inputs, max_length=150, num_beams=4, early_stopping=True)


    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Aşama 3: Sistem Entegrasyonu ve Test

In [78]:

def rag_system(user_query, df_logs):

    response = generate_response(df_logs, user_query)
    return response

user_query = "500 hata kodları hakkında bilgi ver"


response = rag_system(user_query, df_logs)
print("Yanıt:", response)

Yanıt: :07:39 - DELETE /contact HTTP/1.1 - 400 01/Sep/2024:13:41:24 - PUT /api/data HTTP/1.1 - 404 01/Sep/2024:13:41:24 - POST /products/item1 HTTP/1.1 - 400 01/Sep/2024:08:07:14 - PUT /signup HTTP/1.1 - 400 01/Sep/2024


In [79]:
def command_line_interface():
    print("Web Trafik Loglarına Dayalı Soru-Cevap Sistemi")
    print("Çıkmak için 'exit' yazın.")

    while True:

        user_query = input("\nSorunuzu yazın: ")


        if user_query.lower() in ['exit', 'quit']:
            print("Çıkılıyor...")
            break

        response = rag_system(user_query,df_logs)


        print("\nYanıt:", response)


command_line_interface()


Web Trafik Loglarına Dayalı Soru-Cevap Sistemi
Çıkmak için 'exit' yazın.

Sorunuzu yazın: Hangi HTTP yöntemi en az kullanıldı?

Yanıt: :14 - PUT /api/data HTTP/1.1 - 404 01/Sep/2024:13:41:24 - POST /products/item1 HTTP/1.1 - 400 01/Sep/2024:08:07:14 - PUT /signup HTTP/1.1 - 404 01/Sep/2024:13:41:24 - POST /products HTTP/1.1 - 400 01/Sep/2024:

Sorunuzu yazın: En fazla PUT isteği hangi syfaya yapıldı?

Yanıt: : - PUT /api/data HTTP/1.1 - 404 01/Sep/2024:13:41:24 - POST /products/item1 HTTP/1.1 - 401 01/Sep/2024:08:07:14 - POST /api/data HTTP/1.1 - 404 31/Aug/2024:13:45 - POST /about HTTP/1.1 - 503 31/Aug/2024:

Sorunuzu yazın: exit
Çıkılıyor...


In [80]:

test_queries = [
    "503 status code veren sayfalar nelerdir?",
    "En fazla DELETE isteği hangi sayfaya yapıldı?",
    "En yüksek yanıt boyutuna sahip istek hangi sayfaya yapıldı?",
    "En fazla POST isteği hangi sayfaya yapıldı?",
    "En fazla PUT isteği hangi sayfaya yapıldı?",
    "Son 10 istekte en fazla hangi sayfa ziyaret edildi?",
    "En fazla 201 durum kodu hangi sayfaya verildi?",
    "Hangi HTTP yöntemi en az kullanıldı?",
]



print("Test Senaryoları Başlatılıyor...\n")

for query in test_queries:
    print(f"Soru: {query}")
    response = rag_system(query, df_logs)
    print("Yanıt:", response)
    print("\n" + "-"*50 + "\n")

print("Testler Tamamlandı.")


Test Senaryoları Başlatılıyor...

Soru: 503 status code veren sayfalar nelerdir?
Yanıt: :14 - PUT /api/data HTTP/1.1 - 404 01/Sep/2024:13:41:24 - POST /products/item1 HTTP/1.1 - 401 01/Sep/2024:08:07:14 - PUT /signup HTTP/1.1 - 404 01/Sep/2024:13:41:24 - POST /products HTTP/1.1 - 400 01/Sep/2024

--------------------------------------------------

Soru: En fazla DELETE isteği hangi sayfaya yapıldı?
Yanıt: : - PUT /api/data HTTP/1.1 - 404 01/Sep/2024:13:41:24 - PUT /signup HTTP/1.1 - 404 01/Sep/2024:07:41:24 - PUT /signup HTTP/1.1 - 404 01/Sep/2024:07:41:24 - POST /api/data HTTP/1.1 - 404 31/Aug/2024:21

--------------------------------------------------

Soru: En yüksek yanıt boyutuna sahip istek hangi sayfaya yapıldı?
Yanıt: : lgili log kaytlar: 01/Sep/2024:13:41:24 - PUT /api/data HTTP/1.1 - 404 01/Sep/2024:07:41:24 - POST /products/item1 HTTP/1.1 - 401 01/Sep/2024:08:07:14 - DELETE /login HTTP/1.1 - 400 01/Sep/2024

--------------------------------------------------

Soru: En fazla 

Aşama 4: Performans Değerlendirmesi

In [81]:
evaluation_queries = {
    "503 status code veren sayfalar nelerdir?": "/api/data",
    "En fazla DELETE isteği hangi sayfaya yapıldı?": "/api/data",
    "En yüksek yanıt boyutuna sahip istek hangi sayfaya yapıldı?": "/api/data",
    "En fazla POST isteği hangi sayfaya yapıldı?": "/products",
    "En fazla PUT isteği hangi sayfaya yapıldı?": "/api/data",
    "Son 10 istekte en fazla hangi sayfa ziyaret edildi?": "/contact",
    "En fazla 201 durum kodu hangi sayfaya verildi?": "/api/data",
    "Hangi HTTP yöntemi en az kullanıldı?": "PUT",

}








correct_answers = 0
total_queries = len(evaluation_queries)

print("Doğruluk Değerlendirmesi Başlıyor...\n")

for query, expected_answer in evaluation_queries.items():
    print(f"Soru: {query}")
    response = rag_system(query, df_logs)
    print("Yanıt:", response)


    if expected_answer.lower() in response.lower():
        print("Doğru")
        correct_answers += 1
    else:
        print("Yanlış")

    print("\n" + "-"*50 + "\n")

accuracy = correct_answers / total_queries
print(f"Doğruluk: {accuracy * 100:.2f}%")


Doğruluk Değerlendirmesi Başlıyor...

Soru: 503 status code veren sayfalar nelerdir?
Yanıt: :14 - PUT /api/data HTTP/1.1 - 404 01/Sep/2024:13:41:24 - POST /products/item1 HTTP/1.1 - 401 01/Sep/2024:08:07:14 - PUT /signup HTTP/1.1 - 404 01/Sep/2024:13:41:24 - POST /products HTTP/1.1 - 400 01/Sep/2024
Doğru

--------------------------------------------------

Soru: En fazla DELETE isteği hangi sayfaya yapıldı?
Yanıt: : - PUT /api/data HTTP/1.1 - 404 01/Sep/2024:13:41:24 - PUT /signup HTTP/1.1 - 404 01/Sep/2024:07:41:24 - PUT /signup HTTP/1.1 - 404 01/Sep/2024:07:41:24 - POST /api/data HTTP/1.1 - 404 31/Aug/2024:21
Doğru

--------------------------------------------------

Soru: En yüksek yanıt boyutuna sahip istek hangi sayfaya yapıldı?
Yanıt: : lgili log kaytlar: 01/Sep/2024:13:41:24 - PUT /api/data HTTP/1.1 - 404 01/Sep/2024:07:41:24 - POST /products/item1 HTTP/1.1 - 401 01/Sep/2024:08:07:14 - DELETE /login HTTP/1.1 - 400 01/Sep/2024
Doğru

---------------------------------------------